## Representación del texto: Word Embeddings

- ### **Word embeddings** son representaciones numéricas densas y continuas de palabras en un espacio vectorial.
- ### Estas representaciones capturan relaciones semánticas y sintácticas entre palabras.
- ### Palabras con significados similares están más cercanas en el espacio vectorial.
- ### Densidad: Cada palabra se representa como un vector en un espacio de dimensiones reducidas (por ejemplo, 100 o 300 dimensiones).
- ### Diferente a las representaciones como las matrices dispersas en el modelo de "bolsa de palabras".
- ### Similitud semántica: Las palabras con significados similares tendrán vectores cercanos en el espacio vectorial.
- ### Por ejemplo, en un buen modelo de embeddings, los vectores de "rey" y "reina" estarán cerca.
- ### Relaciones semánticas y aritmética vectorial:
- ### Se pueden realizar operaciones matemáticas que reflejan relaciones semánticas, como:
- ### **rey−hombre+mujer≈reina**



## Enfoques Word Embeddings:
- #### los embeddings generalmente se entrenan a partir de grandes cantidades de texto utilizando algoritmos que buscan capturar las co-ocurrencias de palabras en un contexto dado. 
- #### Algunos métodos populares son:
- #### **Word2Vec**:
   - #### Utiliza dos enfoques: Skip-Gram (predice el contexto dada una palabra) y CBOW (predice una palabra dado su contexto).
- #### **GloVe** (Global Vectors for Word Representation):
    - #### Basado en una matriz de co-ocurrencia de palabras en un corpus grande.
    - #### Intenta capturar la probabilidad relativa de dos palabras que co-ocurren.
- #### **FastText**:
    - #### Similar a Word2Vec, pero considera subpalabras (caracteres), lo que mejora la representación de palabras raras o con errores ortográficos.
- #### **Contextuales (p. ej., BERT, GPT)**:
    - #### Modelos que generan representaciones de palabras dependiendo del contexto en el que aparecen.


# Modelos FastText

### [https://fasttext.cc](https://fasttext.cc)

## Modelo pre-entrenados para el idioma español

### [https://fasttext.cc/docs/en/crawl-vectors.html](https://fasttext.cc/docs/en/crawl-vectors.html#models)


## Modelo pre-entrenados para diferentes regiones del idioma español

### [https://ingeotec.github.io/regional-spanish-models](https://ingeotec.github.io/regional-spanish-models/#resources)




## Instalación del paquete FastText. 
### Se recomienda la instalación en sistemas operativos linux o mac os.


In [ ]:
!pip install fasttext

## Cargar el modelo pre-entrenado para la codificación de word embeddings

In [ ]:
import fasttext

# Descargar el modelo para el español de la página de FastText
ft = fasttext.load_model('/Volumes/data/temp/cc.es.300.bin')

## Obtener el vector de una palabra

In [ ]:

#Obtención del vector de una palabra de palabras
print(ft.get_word_vector("hola"))

# equivalente 
# Vector Denso de la palabra "hola"
print(ft["hola"])


## Total de palabras en el modelo

In [ ]:
# Obtiene la lista total de palabras del modelo
# ft.get_words()

# Equivalente a la propiedad words

# Obtención el total del vocabulario
print("total de palabras: ", len(ft.words))

#primeras 10 palabras del vocabulario
ft.words[:10]

## Codificar oraciones en su forma de embeddings

In [ ]:
# Obtiene la representación de embedding de la oración

vec = ft.get_sentence_vector("hola me siento muy feliz")
print(vec)

## Obtención de las palabras vecinas más cercanas basadas en vectores densos

In [ ]:
ft.get_nearest_neighbors("mareado")

# Operaciones con vectores semánticos

In [ ]:
X= (ft.get_word_vector("rey") - ft.get_word_vector("hombre")) + ft.get_word_vector("mujer")

X

# Operaciones con vectores semánticos: Analogías

In [ ]:
analogia = ft.get_analogies("rey","hombre", "mujer")
print(analogia)

# Procesando archivos que ya contiene su representación en word enbeddings

In [ ]:
import pandas as pd
print(f"Cargando datos...")
dataset_train = pd.read_json("./data/dataset_polaridad_es_train_embeddings.json", lines=True)
dataset_test = pd.read_json("./data/dataset_polaridad_es_test_embeddings.json", lines=True)


In [ ]:
# datos del train
dataset_train.head()


In [ ]:
# datos del test
dataset_test.head()

# Entrenando al clasificador

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import svm
from sklearn.metrics import f1_score
import numpy as np

LEncoder = LabelEncoder()

# El texto ya está en su forma vectorial calculado por los word embeddings del archivo según el campo
# Campo = Vector de Word Embeddings de 300 dimensiones
# we_ft = Word Embeddings calculados de textos generales del español 
# we_mx = Word Embeddings calculados de textos del español de México
# we_es = Word Embeddings calculados de textos del español de España

X_train_text = dataset_train['text'].to_numpy()
# Convertir a matriz de arrays de numpy
X_train = np.vstack(dataset_train["we_ft"].to_numpy())
Y_train = dataset_train['klass'].to_numpy()

X_test_text = dataset_test['text'].to_numpy()
# Convertir a matriz de arrays de numpy
X_test = np.vstack(dataset_test["we_ft"].to_numpy())
Y_test = dataset_test['klass'].to_numpy()

Y_train_encoded= LEncoder.fit_transform(Y_train)
Y_test_encoded= LEncoder.transform(Y_test)


# clf = svm.LinearSVC() = svm.SVC(kernel='linear')  son equivalentes los dos modelos.  
# kernel{‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’}
clf = svm.SVC(kernel='linear')
# clf = svm.SVC(kernel='rbf')

# Entrenar el modelo con los word embeddings
clf.fit(X_train, Y_train_encoded)
# Predecir los datos con el modelo entrenado para el conjunto de test
y_pred = clf.predict(X_test)
# Evaluar el desempeño de acuerdo a la métrica f1-macro
score = f1_score(Y_test_encoded, y_pred, average="macro")
print(f"Desempeño del modelo en el conjunto de test: {score}")

# **Ejercicio 1**
### Entrenar y evaluar el desempeño del clasificador entrenado en el conjunto de test.

### 1. Probar con diferentes kernels del clasificador Support Vector Machine (SVM)
- ### lineal, rbf, sigmoid, poly.
### 2. Probar diferentes tipos de word embeddings previamente codificados:
- ### we_ft (General) , we_mx (México), we_es (España) 


## **Ejercicio 2**
- ### Modificar el algoritmo genético definido en el archivo $07\_GA\_clasificacion\_texto\_SVM.py$ para implementar el fitness con el desempeño del clasificador usando representación de word embeddings y el mejor kernel identificado para la predicción de datos con word embeddings